In [ ]:
#step-1
!pip install flask pyngrok
!pip install pydub
!pip install flask-cors


In [ ]:
# 1. Cleanly reinstall numpy
!pip install numpy==1.26.4 --force-reinstall

# 2. Install compatible TensorFlow and TensorBoard
!pip install tensorflow==2.15.0 tensorboard==2.15.1

# 3. (Optional) If you're using audiocraft (MusicGen)
!pip install -U git+https://github.com/facebookresearch/audiocraft

# 4. Other stuff you might need
!pip install flask flask-cors pyngrok pydub requests


In [ ]:
#step-3
from flask import Flask, request, send_file,Response
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch
import tempfile
import os
from pyngrok import ngrok
import io
from pyngrok import ngrok



In [ ]:
!ngrok config add-authtoken '2s6uFeC6pTxNdVGPQdSoRbhr9c1_6PeaKNh87o5j8vEsyx4or'


In [ ]:
import numpy as np
import logging
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
from pyngrok import ngrok
import base64
import requests

# Initialize Flask app
app = Flask(__name__)
logging.basicConfig(level=logging.DEBUG)

# Enable CORS
CORS(app, resources={r"/*": {"origins": "*"}})

# Set up ngrok tunnel
port = 8000
public_url = ngrok.connect(port).public_url
print(f"Colab Public URL: {public_url}")

# Send the ngrok URL to the fixed backend server
fixed_backend_url = "https://3caf-2400-3680-10b-66-f55b-681-8ec4-cbe.ngrok-free.app/"  # Replace with your backend's fixed URL
try:
    response = requests.post(f"{fixed_backend_url}/update_colab_ngrok_url", json={"colabNgrokUrl": public_url})
    if response.ok:
        print("Successfully updated the fixed backend server with the Colab ngrok URL.")
    else:
        print(f"Failed to update the backend server: {response.status_code}, {response.text}")
except Exception as e:
    print(f"Error sending ngrok URL to backend server: {e}")

# Load the MusicGen model
model = musicgen.MusicGen.get_pretrained('melody', device='cuda')
model.set_generation_params(duration=30)  # Default duration

@app.route('/generate', methods=['POST', 'OPTIONS'])
@cross_origin()
def generate_music():
    if request.method == 'OPTIONS':
        return jsonify({"message": "CORS preflight successful"}), 200

    data = request.json
    description = data.get("description", "")
    duration = int(data.get("time", 30))  # Default duration to 30 if not provided

    if not description or not duration:
        return jsonify({"error": "Description and time are required!"}), 400

    try:
        logging.info(f"Generating music with description: {description} and duration: {duration}")

        # Dynamically set the duration before generating music
        model.set_generation_params(duration=duration)

        res = model.generate([description], progress=False)

        # Convert tensor to NumPy array and then bytes
        audio_array = res[0].cpu().numpy()
        audio_bytes = audio_array.astype(np.float32).tobytes()

        # Send audio to the backend server
        def send_audio_to_server(audio, sample_rate):
            audio_base64 = base64.b64encode(audio).decode('utf-8')
            url = f"{fixed_backend_url}/upload_audio"
            data = {
                'audio': audio_base64,
                'sample_rate': sample_rate
            }
            response = requests.post(url, json=data)
            return response.json()

        response = send_audio_to_server(audio_bytes, 32000)
        logging.info(f"Response from backend server: {response}")

        return jsonify(response), 200

    except Exception as e:
        logging.error(f"Error generating music: {str(e)}")
        return jsonify({"error": f"Error generating music: {str(e)}"}), 500

app.run(port=port)


Colab Public URL: https://8625-34-16-143-147.ngrok-free.app
Successfully updated the fixed backend server with the Colab ngrok URL.


/usr/local/lib/python3.11/dist-packages/audiocraft/models/musicgen.py:83: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-melody
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Apr/2025 10:49:33] "OPTIONS /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2025 10:49:56] "POST /generate HTTP/1.1" 200 -
